In [1]:
#Import earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yrgsvigOkzswXPF_7J_zgfzEATBz3Ecrp2BrGsxAw8A&tc=xhJG3A9aTPSuA9hIh88K5l6NF0Ca5-7Hri_d3QwmSA4&cc=x1KlyLmPsEK-7sft4vrG0BTIbml8r-tDc5wxjzDQjnY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhV2wpVI7cxI7QvBwfzqX9ftVJSrS54BUkc-EkgH3369AcFgdrHmnGU

Successfully saved authorization token.


In [9]:
#Get a feature collection with Egypt
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Turkey = countries.filter(ee.Filter.eq('country_na', 'Turkey'))

# Load point
point1 = ee.Geometry.Point([35, 7])
poin2 = ee.Geometry.Point([39, 12])

#Compute a buffer of the polygon
forest1 = point1.buffer(1000)
nonforest1 = point1.buffer(1000)

# Input imagery is a cloud free Landsat 8 composite
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1")

image = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': l8.filterDate('2018-01-01', '2018-12-31'),
     'asFloat': True
})

In [10]:
# Use these bands for prediction
bands = ['B1', 'B2', 'B3', 'B4', 'B5','B6','B7', 'B10', 'B11']

In [11]:
# Make a feature collection from the hand-made geometries
polygons = ee.FeatureCollection([
  ee.Feature(nonforest1, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
])

In [12]:
#Get the values for all the pixels in each polygon in the training
training = image.sampleRegions(**{
    # Get the sample form the polygons FeatureCollection
    'collection': polygons,
    # Keep this list of properties from the polygons
     'properties': ['class'],
      #Set the scale to get Landsat pixels in the polygons
      'scale': 30
})

In [13]:
# Create an SVM classifier with custom parameters
classifier = ee.Classifier.libsvm(**{
    'kernelType': 'RBF',
    'gamma': 0.5,
    'cost': 10
})

In [14]:
# Train the classifier
trained = classifier.train(training, 'class', bands)

In [15]:
# Classify the image
classified = image.classify(trained)

# Clip image
clip = classified.clip(Turkey)

In [17]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[9, 39], zoom_start = 8)

# Add the layer to the map object.
#my_map.add_ee_layer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.5, 'gamma': 2}, "image")
my_map.add_ee_layer(clip, {'min': 0, 'max': 1, 'palette': ['green', 'red']}, 'deforestation')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)